Import dependecies

In [116]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib
from urllib.request import Request, urlopen, urlparse
import re
import time
import requests
import json
import analytics

URL to scrapp and Headers to avoid blocking from kickstarter

In [48]:
url ='https://www.kickstarter.com/discover/advanced?raised=2&sort=end_date&seed=2743718&page=1'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

In [49]:
def engine(pages) : 
    url = url ='https://www.kickstarter.com/discover/advanced?raised=2&sort=enddate&seed=2743718&page='
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    lst = []
    for i in range(1,pages) :
        r = requests.get(url+str(i),headers=headers)
        soup = BeautifulSoup(r.content, 'html.parser')
        lst.append(soup)
        i =+ 1   
    return lst

Actually not a scrapper but a function to extract and read infos from raw html

In [50]:
def scraper_prototype(text_profile, prefix):

    if '"name"' in text_profile:
        text_profile = re.sub('"', '', text_profile)

    text2 = text_profile.split(',')
    dummy2 = [re.findall('[A-Za-z0-9_]+:', ww) for ww in text2]
    itemname = [dum_part[0] for dum_part in dummy2 if len(dum_part) > 0]
    ans_array = [re.search(xxy1+'(.*?)'+xxy2, text_profile).group(1).rstrip(',')
                 for xxy1, xxy2 in zip(itemname[:-1], itemname[1:])]
    itemnames = [x.rstrip(':') for x in itemname]

    info_dict = dict()
    for item, ans in zip(itemnames, ans_array):
        info_dict[prefix+'_'+item] = ans
    return info_dict

Function to transform result into a string

In [51]:
def to_string(container) : 
    whole_string = str(container).replace('&amp', '').replace('&quot;', '')
    return whole_string

Function to extract name and basic project infos

In [52]:
def get_name(whole_string) : 
    if '"name"' in whole_string:
        project_info = '"name"' + re.findall('"name"(.+?)"creator"', whole_string)[0]
    else:
        project_info = 'name' + re.findall('name(.+?)creator', whole_string)[0]
    project_info_dict = scraper_prototype(project_info, 'project')
    df1 = pd.DataFrame.from_dict(project_info_dict, orient='index').transpose()
    return df1

Function to extract Creator infos 

In [53]:
def get_creator(whole_string) : 
    if '"creator"' in whole_string:
        creator_string = re.findall(r'\"creator\"\:\{(.+?)\}', whole_string)[0]
    else:
        creator_string = re.findall(r'creator\:\{(.+?)\}', whole_string)[0]
    creator_info_dict = scraper_prototype(creator_string, 'creator')
    creator_info_dict['creator_slug'] = creator_info_dict.get('creator_slug', 'Null')
    df2 = pd.DataFrame.from_dict(creator_info_dict, orient='index').transpose()
    
    return df2

Function to get location

In [54]:
def get_location(whole_string) : 
    if '"location"' in whole_string:
        location_string = re.findall(r'\"location\"\:\{(.+?)\}', whole_string)[0]
    else:
        location_string = re.findall(r'location\:\{(.+?)\}', whole_string)[0]
    location_info_dict = scraper_prototype(location_string, 'location')
    df3 = pd.DataFrame.from_dict(location_info_dict, orient='index').transpose()
    return df3

Fucntion to concatenate all the dataframe from previous function into one

In [55]:
def make_a_row(df1,df2,df3) : 
    df = pd.concat((df1, df2, df3), axis=1)
    return df

Function to actually make a row for each project

In [56]:
def row_flow(container) : 
    string = to_string(container)
    name = get_name(string)
    creator = get_creator(string)
    location = get_creator(string)
    #profile = get_profile(string)
    row = make_a_row(name,creator,location)
    return row

Function to check if the url is a valid url

In [57]:
def is_valid(url):
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)

Actual scrapper (print a message if we are actually scrapping or if something went wrong)

In [58]:
r = requests.get(url, headers=headers)
soup = BeautifulSoup(r.content, 'html.parser')
if r.status_code == 200 : 
    print('Reading the Website')
else : 
    print('Something went wrong')

Reading the Website


Divide the raw html into containers from each project-card

In [59]:
containers = soup.find_all('div', class_="js-react-proj-card")

Create row and concatenate them

In [60]:
row = row_flow(containers[0])
row0 = row_flow(containers[1])
row1 = row_flow(containers[2])
row2 = row_flow(containers[3])
row3 = row_flow(containers[4])
row4 = row_flow(containers[5])
row5 = row_flow(containers[6])
row6 = row_flow(containers[7])
row7 = row_flow(containers[8])
row8 = row_flow(containers[9])
row9 = row_flow(containers[10])
row10 = row_flow(containers[11])

In [73]:
df = pd.concat([row, row2, row3, row4, row5, row6, row7, row8, row10],ignore_index=True)

In [74]:
df

,project_name,project_blurb,project_goal,project_pledged,project_state,project_slug,project_disable_communication,project_country,project_country_displayable_name,project_currency,...,creator_small,creator_id,creator_name,creator_slug,creator_is_registered,creator_is_email_verified,creator_chosen_currency,creator_is_superbacker,creator_avatar,creator_small
0,Make 100: Physics 1885,Restoring a forgotten scientific atlas from th...,1300.0,13378.0,live,make-100-physics-1885,false,ES,Spain,EUR,...,https://ksr-ugc.imgix.net/assets/010/365/358/1...,413661267,Alejandro Polanco,alpoma,null,null,null,null,{thumb:https://ksr-ugc.imgix.net/assets/010/36...,https://ksr-ugc.imgix.net/assets/010/365/358/1...
1,Shimmering Basin #2,Shimmering Basin #2 returns the reader to Shje...,2000.0,2036.29,live,shimmering-basin-2,false,CA,Canada,CAD,...,https://ksr-ugc.imgix.net/assets/030/124/176/7...,1401550725,Braedan Hafichuk,braedanhafichuk,null,null,null,null,{thumb:https://ksr-ugc.imgix.net/assets/030/12...,https://ksr-ugc.imgix.net/assets/030/124/176/7...
2,Starless Vampire Music Film,Brianna Knickerbocker's known for her voice ac...,22999.0,23348.0,live,starless-music-videos,false,US,the United States,USD,...,https://ksr-ugc.imgix.net/assets/035/794/750/1...,200708175,Starless Music,starlessmusic,null,null,null,null,{thumb:https://ksr-ugc.imgix.net/assets/035/79...,https://ksr-ugc.imgix.net/assets/035/794/750/1...
3,LINE ; MASS - Artbook by Alberto Ruiz,The new and massive collection of over 400 pag...,20000.0,111219.0,live,lineandmass,false,FR,France,EUR,...,https://ksr-ugc.imgix.net/assets/008/446/593/5...,1580441778,Spiridon,spiridon,null,null,null,null,{thumb:https://ksr-ugc.imgix.net/assets/008/44...,https://ksr-ugc.imgix.net/assets/008/446/593/5...
4,Goblin Civil War,An original set of rules and miniatures for ga...,1000.0,1000.0,live,goblin-civil-war,false,GB,the United Kingdom,GBP,...,https://ksr-ugc.imgix.net/assets/010/366/868/2...,179272384,Chris Abbey @ Sally 4th,blackstone,null,null,null,null,{thumb:https://ksr-ugc.imgix.net/assets/010/36...,https://ksr-ugc.imgix.net/assets/010/366/868/2...
5,Danielle Duval - Le Ciel est Toilé,"Voici le projet de ma mère, Danielle Duval. Da...",10000.0,11337.29,live,danielle-duval-le-ciel-est-toile,false,CA,Canada,CAD,...,https://ksr-ugc.imgix.net/assets/036/225/053/7...,1743988493,Luc Laberge,danielleduvalart,null,null,null,null,{thumb:https://ksr-ugc.imgix.net/assets/036/22...,https://ksr-ugc.imgix.net/assets/036/225/053/7...
6,"Fresh Produce Stationary: Pins, Tapes, Sticker...","Adorable and fresh, these fruits and veggies m...",2000.0,3291.0,live,fresh-produce-stationary-pins-tapes-stickers-a...,false,US,the United States,USD,...,https://ksr-ugc.imgix.net/assets/030/400/335/7...,643370180,Stars and Sprinkles,starsandsprinkles,null,null,null,null,{thumb:https://ksr-ugc.imgix.net/assets/030/40...,https://ksr-ugc.imgix.net/assets/030/400/335/7...
7,RB White - Comic book covers AGAIN,2nd Comic book with variant cover illustration...,3000.0,15473.29,live,rb-white-comic-book-covers-again,false,CA,Canada,CAD,...,https://ksr-ugc.imgix.net/assets/006/937/139/2...,58475264,RB White,rbwhite,null,null,null,null,{thumb:https://ksr-ugc.imgix.net/assets/006/93...,https://ksr-ugc.imgix.net/assets/006/937/139/2...
8,Pocket Tony Enamel Pin Series,"Tony, Tony, and more Tony ✧･ﾟ",900.0,14982.0,live,pocket-tony-enamel-pin-series,false,US,the United States,USD,...,https://ksr-ugc.imgix.net/assets/032/235/715/7...,396652360,wspace,wspace,null,null,null,null,{thumb:https://ksr-ugc.imgix.net/assets/032/23...,https://ksr-ugc.imgix.net/assets/032/235/715/7...


Now that we have a dataframe we drop duplicates and useless columns

In [75]:
df = df.loc[:,~df.columns.duplicated()]

In [76]:
df.drop(['project_disable_communication','project_country_displayable_name',
         'project_currency_symbol','project_current_currency',
         'project_currency_trailing_code','project_staff_pick','project_backers_count',
         'project_static_usd_rate','project_converted_pledged_amount','project_fx_rate',
         'project_usd_exchange_rate','creator_is_registered','creator_is_email_verified',
         'creator_chosen_currency','creator_is_superbacker','creator_avatar','project_usd_pledged','project_is_starrable',
        'creator_small','creator_slug','project_currency'],axis=1,inplace=True)

/var/folders/q1/v8q8_chd1rd46t7djqglmp6r0000gn/T/ipykernel_42419/2027654935.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['project_disable_communication','project_country_displayable_name',


In [77]:
df

,project_name,project_blurb,project_goal,project_pledged,project_state,project_slug,project_country,project_deadline,project_state_changed_at,project_created_at,project_launched_at,creator_id,creator_name
0,Make 100: Physics 1885,Restoring a forgotten scientific atlas from th...,1300.0,13378.0,live,make-100-physics-1885,ES,1646409238,1643817239,1643207633,1643817238,413661267,Alejandro Polanco
1,Shimmering Basin #2,Shimmering Basin #2 returns the reader to Shje...,2000.0,2036.29,live,shimmering-basin-2,CA,1646412346,1643820347,1642087289,1643820346,1401550725,Braedan Hafichuk
2,Starless Vampire Music Film,Brianna Knickerbocker's known for her voice ac...,22999.0,23348.0,live,starless-music-videos,US,1646413339,1643821341,1638233528,1643821339,200708175,Starless Music
3,LINE ; MASS - Artbook by Alberto Ruiz,The new and massive collection of over 400 pag...,20000.0,111219.0,live,lineandmass,FR,1646413371,1643821373,1641207976,1643821371,1580441778,Spiridon
4,Goblin Civil War,An original set of rules and miniatures for ga...,1000.0,1000.0,live,goblin-civil-war,GB,1646416800,1644858202,1619856514,1644858201,179272384,Chris Abbey @ Sally 4th
5,Danielle Duval - Le Ciel est Toilé,"Voici le projet de ma mère, Danielle Duval. Da...",10000.0,11337.29,live,danielle-duval-le-ciel-est-toile,CA,1646417114,1643825115,1643659246,1643825114,1743988493,Luc Laberge
6,"Fresh Produce Stationary: Pins, Tapes, Sticker...","Adorable and fresh, these fruits and veggies m...",2000.0,3291.0,live,fresh-produce-stationary-pins-tapes-stickers-a...,US,1646418527,1644258528,1642638216,1644258527,643370180,Stars and Sprinkles
7,RB White - Comic book covers AGAIN,2nd Comic book with variant cover illustration...,3000.0,15473.29,live,rb-white-comic-book-covers-again,CA,1646419124,1643827125,1643333250,1643827124,58475264,RB White
8,Pocket Tony Enamel Pin Series,"Tony, Tony, and more Tony ✧･ﾟ",900.0,14982.0,live,pocket-tony-enamel-pin-series,US,1646420400,1645208740,1643088761,1645208739,396652360,wspace


Now we go to each creator and take all the link from each page, no selection is made at this point

In [78]:
lst =[]
my_dict ={}
for i in range(0,len(df)) :
    creator_id = str(df.loc[i]['creator_id'])
    html = requests.get('https://www.kickstarter.com/profile/'+creator_id+'/about', headers=headers)
    soup = BeautifulSoup(html.content, 'html.parser')
    for a in soup.find_all('a', href=True):
        #print(str(df.loc[i]['creator_id']))
        lst.append((a['href'], creator_id ))
        my_dict[creator_id]=(a['href'])

Now we want to keep just real website and remove all other links

In [79]:
lst2 = []
for el in lst : 
    if is_valid(el[0]) != True :
        lst.remove(el)
    if el[0].find('http') != -1 :
        lst2.append(el)

Now that we have real website, we check every one to see if the shopify variable is in it, if not we remove it from the list

In [80]:
s = []
for el in  lst2: 
    try :
        r = requests.get(el[0])
        t = str(r.text)
        shopify = t.find('shopify')
        s.append((shopify,el))
    except requests.exceptions.SSLError:
        pass

In [91]:
s

[(-1, ('https://maptorian.com', '413661267')),
 (-1, ('http://albedopatrol.com', '179272384')),
 (1873, ('https://starsandsprinkles.shop', '643370180'))]

In [95]:
last_lst = []
for el in s :
    if el[0] < 0 :
        s.remove(el)
    else : 
        last_lst.append(el)

In [99]:
len(s)

1

In [100]:
if len(s) == 0 : 
    print('No shopify user for now')
else : 
    print(len(s),' Shopify user detected')

1  Shopify user detected


In [78]:
print(last_lst)

[]


In [101]:
df.set_index("creator_id", inplace = True)

Now that we have all the shopify website, we get back all the infos from the previous DF (i use the creator_id as an index)
and we create a Dataframe with all the infos and the valid website

In [105]:
for el in s : 
    infos = el[1]

In [106]:
new_df = df.loc[infos[1]]

In [107]:
new_df['website'] = infos[0]

In [108]:
new_df.columns = new_df[0]

In [109]:
new_df.columns = df.iloc[:, 0]

In [110]:
print(new_df)

project_name                Fresh Produce Stationary: Pins, Tapes, Sticker...
project_blurb               Adorable and fresh, these fruits and veggies m...
project_goal                                                           2000.0
project_pledged                                                        3291.0
project_state                                                            live
project_slug                fresh-produce-stationary-pins-tapes-stickers-a...
project_country                                                            US
project_deadline                                                   1646418527
project_state_changed_at                                           1644258528
project_created_at                                                 1642638216
project_launched_at                                                1644258527
creator_name                                              Stars and Sprinkles
website                                        https://starsands

In [114]:
result = new_df.to_json(orient="index", indent=4)
print(result)

{
    "project_name":"Fresh Produce Stationary: Pins, Tapes, Stickers ; More!",
    "project_blurb":"Adorable and fresh, these fruits and veggies merch will brighten your day (and maybe your salad too)",
    "project_goal":"2000.0",
    "project_pledged":"3291.0",
    "project_state":"live",
    "project_slug":"fresh-produce-stationary-pins-tapes-stickers-and-more",
    "project_country":"US",
    "project_deadline":"1646418527",
    "project_state_changed_at":"1644258528",
    "project_created_at":"1642638216",
    "project_launched_at":"1644258527",
    "creator_name":"Stars and Sprinkles",
    "website":"https:\/\/starsandsprinkles.shop"
}


And we save everything into a json document

In [117]:
with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=4)

In [133]:
import uuid

In [118]:
APIKEY ='Q6QwJI6Zo7Alfp5UxKoKncwh0xu3nk2O'

In [130]:
analytics.write_key = APIKEY

In [122]:
with open("data.json") as jsonFile:
    jsonObject = json.load(jsonFile)
    jsonFile.close()

In [127]:
j = json.loads(jsonObject)

AttributeError: 'dict' object has no attribute 'key'

In [149]:
project_name = j['project_name']
creator_name = j['creator_name']
blurb = j['project_blurb']
project_pledged = j['project_pledged']
project_goal = j['project_goal']
website = j['website']

In [ ]:
analytics.track('f4ca124298', 'Article Bookmarked', {
    'title': 'Snow Fall',
    'subtitle': 'The Avalance at Tunnel Creek',
    'author': 'John Branch'
})

In [145]:
f = str(uuid.uuid1())

In [146]:
call_name = 'Kickstart-Completed'

'a2c61250-9bd7-11ec-9f6a-0e286d93875d'

In [ ]:
analytics.track(str(uuid.uuid1()), call_name,{
    'Project Name': project_name,
    'Creator Name' : creator_name,
    'blurb' : blurb,
    'Project Goal' : project_goal,
    'Project Pledged':project_pledged,
    'Website':
} )

In [142]:
n

UUID('6c52be9e-9bd7-11ec-9f6a-0e286d93875d')

UUID('6c52be9e-9bd7-11ec-9f6a-0e286d93875d')